In [35]:
import numpy as np

####  $\Large{\alpha_{t,i}}$ 

In [36]:

def sum_obs_1st_part(b,z,rho,n_states = None,n_observations = None):
    #b np.array shape-> (states,observation)
    #z np.array shape (observation, 1)
    #rho np.array shape (states,1)
    b = b.reshape(n_states, n_observations)
    z = z.reshape(n_observations,)
    rho = rho.reshape(n_observations,)
    return np.dot(b,z*rho)

def sum_obs_nd_part(b,b_obs_col,z,rho, n_states = None, n_observations =None):
    #b np.array shape-> (states,observation)
    #z np.array shape (observation 1)
    #rho np.array shape (states,1)
    b = b.reshape(n_states, n_observations)
    z = z.reshape(n_observations, )
    rho = rho.reshape(n_observations, )
    b_col_matrix = (np.ones(n_states*n_observations).reshape(n_states,n_observations)) * b[:,b_obs_col].reshape(n_states,1)
    return np.dot(b_col_matrix,z*(1-rho))

def compute_complete_obs_sum(b, b_obs_col,z,rho, n_states =None , n_observations= None):
    fst_part =  sum_obs_1st_part(b,z,rho, n_states, n_observations)
    snd_part = sum_obs_nd_part(b, b_obs_col,z,rho, n_states, n_observations)
    return fst_part + snd_part

def compute_log_alpha_1_i(b, b_obs_col, z, rho, pi, n_states, n_observations):
    pi_vector = pi.reshape(1,n_states) 
    alpha_1_i = pi_vector * compute_complete_obs_sum(b, b_obs_col,z,rho, n_states , n_observations)
    return np.log(alpha_1_i)


def compute_A_alpha_minus_prod(alpha_minus_1,A_matrix,n_states):
    #alpha_minus_1: shape ->(n_states,1)
    #A_matrix: shape-> (n_states,n_states)
    #n_states: int
    alpha_minus_1 = alpha_minus_1.reshape(n_states,1)
    A_matrix  = (np.log(A_matrix).reshape(n_states, n_states))
    A_alpha_minus_prod = np.sum(alpha_minus_1 + A_matrix,axis=0)
    return A_alpha_minus_prod

def compute_alpha_t(alpha_minus_1,A_matrix,b, b_obs_col,z,rho, n_states =None , n_observations= None):
    obs_sum = np.log(compute_complete_obs_sum(b, b_obs_col,z,rho, n_states, n_observations))
    A_alpha_minus_prod = compute_A_alpha_minus_prod(alpha_minus_1, A_matrix,n_states)
    return A_alpha_minus_prod + obs_sum

def compute_alpha_t_i(A, B, pi ,z_vector, rho_vector, x_vector, t, n_states_, n_observations_):
    lista_alpha = []
    z_0 = z_vector[0]
    rho_0 = rho_vector[0]
    x_0 = x_vector[0]
    alpha_t_i  = compute_log_alpha_1_i(B, x_0, z_0, rho_0, pi, n_states = n_states_, n_observations = n_observations_)
    lista_alpha.append(list(alpha_t_i[0]))
    for t in range(2,t+1):
        z_i = z_vector[t-1]
        rho_i = rho_vector[t-1]
        x_i  = x_vector[t-1]
        alpha_t_i = compute_alpha_t(alpha_t_i,A,B, x_i, z_i,rho_i, n_states=n_states_, n_observations=n_observations_)
        lista_alpha.append(list(alpha_t_i))
    return np.array(lista_alpha)



$\alpha_{1,i} = \pi_i \big(\sum_{j\in X}b_{i,j}z_{i,j}\rho_{1,j} + b_{1,x_{1}}z_{1,j}(1-\rho_{1,j})\big)$

$\Omega_{t,i}  = \big(\sum_{j \in X}b_{i,j}z_{t,j}\rho_{t,j} + b_{i,x_{t}}z_{t,j}(1-\rho_{t,j})\big)$

$\alpha_{t,i} = \Omega_{t,i} \sum_{k\in Q} \alpha_{t-1,k}a_{k,i}$  

$\log{\alpha_{t,i}} = \log{\Omega_{t,i}} + \sum_{k\in Q} \log{\alpha_{t-1,k}}+\log{a_{k,i}} $

$\alpha_{t =1,i = 1} =\small{\pi_1 \Big(\big(b_{1,1}z_{1,1}\rho_{1,1} + b_{1,x_1 =2}z_{1,2}(1 -\rho_{1,1}) \big)+ \big(b_{1,2}z_{1,2}\rho_{1,2} + b_{1,x_1 =2}z_{1,2}(1 -\rho_{1,2}) \big) + \big(b_{1,3}z_{1,3}\rho_{1,3} + b_{1,x_1 =2}z_{1,3}(1 -\rho_{1,3}) \big) \Big)}$

$\alpha_{t =1,i = 2} =\small{\pi_2 \Big(\big(b_{2,1}z_{1,1}\rho_{1,1} + b_{2,x_1 =2}z_{1,1}(1 -\rho_{1,1}) \big)+ \big(b_{2,2}z_{1,2}\rho_{1,2} + b_{2,x_1 =2}z_{1,2}(1 -\rho_{1,2}) \big) + \big(b_{2,3}z_{1,3}\rho_{1,3} + b_{2,x_1 =2}z_{1,3}(1 -\rho_{1,3}) \big) \Big)}$

$\log{\alpha_{t =2,i = 1}} =\small{\log{\Omega_{2,1}} + \log{\alpha_{1,1}} + \log{a_{1,1} + \log{\alpha_{1,2}} + \log{a_{2,1}}}}$

$\small{\Omega_{2,1} = b_{1,1}z_{2,1}\rho_{2,1} + b_{1,x_{1}}z_{2,1}(1-\rho_{2,1})+ b_{1,2}z_{2,2}\rho_{2,2}+b_{1,x_{1}}z_{2,2}(1-\rho_{2,2})+b_{1,3}z_{2,3}\rho_{2,3}+b_{1,x_{1}}z_{2,3}(1-\rho_{2,3})}$

$\log{\alpha_{t =2,i = 2}} =\small{\log{\Omega_{2,2}} + \log{\alpha_{1,1}} + \log{a_{1,2} + \log{\alpha_{1,2}} + \log{a_{2,2}}} }$

$\small{\Omega_{2,2} = b_{2,1}z_{2,1}\rho_{2,1} + b_{2,x_{1}}z_{2,1}(1-\rho_{2,1})+ b_{2,2}z_{2,2}\rho_{2,2}+b_{2,x_{1}}z_{2,2}(1-\rho_{2,2})+b_{2,3}z_{2,3}\rho_{2,3}+b_{2,x_{1}}z_{2,3}(1-\rho_{2,3})}$


-------------------------------------------------------------------------------------------------------------------





a =  $\left(\begin{array}{cc} 
a_{11} & a_{12}\\
a_{21} & a_{22}
\end{array}\right)$ = $\left(\begin{array}{cc} 
0.4 & 0.6\\
0.7 & 0.3
\end{array}\right)$ 

b =  $\left(\begin{array}{ccc} 
b_{11} & b_{12} & b_{13}\\
b_{21} & b_{22} & b_{23}
\end{array}\right)$ = $\left(\begin{array}{cc} 
0.1 & 0.4 & 0.5\\
0.4 & 0.5 & 0.1
\end{array}\right)$ 

$\pi  = \left(\begin{array}{cc} 
\pi_{1} & \pi_{12} 
\end{array}\right) = \left(\begin{array}{cc} 
0.7 & 0.3
\end{array}\right)$

$x_1 = 2$;
$z_{1,j}  = \left(\begin{array}{ccc} 
z_{11} & z_{12} & z_{13}
\end{array}\right) = \left(\begin{array}{ccc} 
0 & 0 & 1
\end{array}\right)$;  $\rho_{1,j} = \left(\begin{array}{ccc} 
\rho_{11} & \rho _{12} & \rho_{13}
\end{array}\right) = \left(\begin{array}{ccc} 
0 & 1 & 1
\end{array}\right) $ ; 


$\alpha_{t =1,i = 1}$ $ =\small{ 0.7 \Big( \big( 0.1 \times 0 \times 0 + 0.4 \times 0 \times 1 \big) + \big( 0.4 \times 0 \times 1+ 0.4 \times 0 \times 0 \big) +\big( 0.5 \times 1 \times 1 + 0.4 \times 1 \times 0  \big) \big) = 0.7 \times 0.5 = 0.35 }$

$\alpha_{t =1,i = 2}$ $ =\small{ 0.3 \Big( \big( 0.4 \times 0 \times 0 + 0.5 \times 0 \times 1 \big) + \big( 0.5 \times 0 \times 1+ 0.5 \times 0 \times 0 \big) +\big( 0.1 \times 1 \times 1 + 0.5 \times 1 \times 0  \big) \big) = 0.3 \times 0.1 = 0.03}$

$\log{\alpha_{t =1,i = 1}} = \log{0.35} = -1.0498 $

$\log{\alpha_{t =1,i = 2}} = \log{0.01} = -4.6051$

In [38]:
A_matrix = np.array([[0.4,0.6],
                    [0.7, 0.3]])

B_matrix = np.array([[0.1, 0.4,0.5],
                    [0.4,0.5,0.1]])

pi_vector = np.array([0.7, 0.3])

z_vector = np.array([0,0,1])

rho_vector = np.array([0,1,1])

x_0 = 1

In [39]:
sum_obs_1st_part(B_matrix,z_vector,rho_vector,n_states = 2,n_observations = 3)

array([0.5, 0.1])

In [40]:
sum_obs_nd_part(B_h,x_0,z_h,rho_h, n_states = 2, n_observations =3)

array([0., 0.])

$x_2 = 3$ ;
$z_{2,j}  = \left(\begin{array}{ccc} 
z_{21} & z_{22} & z_{23}
\end{array}\right) = \left(\begin{array}{ccc} 
1 & 0 & 0
\end{array}\right)$;  $\rho_{2,j} = \left(\begin{array}{ccc} 
\rho_{21} & \rho _{22} & \rho_{23}
\end{array}\right) = \left(\begin{array}{ccc} 
1 & 1 & 1
\end{array}\right) $ ; 


$\small{\Omega_{2,1} = 0.1 \times 1 \times 1 + 0.1 \times 0 \times 0+ 0.4 \times 0 \times 1 + 0.1 \times 1 \times 0 +0.5 \times 0 \times 1 +0.1 \times 0 \times 0} = 0.1$


$\small{\Omega_{2,2} = 0.4 \times 1 \times 1 + 0.4 \times 1 \times 0 + 0.5 \times 0 \times 1 +0.4 \times 0 \times (1-1)+0.1 \times 0 \times 1+ 0.1 \times 1 \times 0} = 0.4$

$\log_{\alpha_{2,1}} = \log{0.1} +  (\log{0.35} + \log{0.4} + \log{0.03} +\log{0.7}) = -2.3025 -5.82934 = -8.1318 $


$\log_{\alpha_{2,2}} = \log{0.4} + (\log{0.35} + \log{0.6} + \log{0.03} + \log{0.3}) = -0.91629 -6.27117 = -7.18746 $

In [31]:
compute_complete_obs_sum(B_h,x_0,z_h,rho_h, n_states =2 , n_observations= 3)

array([0.1, 0.4])

In [8]:
alpha_1_i  = compute_log_alpha_1_i(B_h, x_0, z_h, rho_h, pi_h, n_states = 2, n_observations = 3)
print(alpha_1_i)

[[-1.04982212 -3.5065579 ]]


In [26]:
np.log(0.1)+np.log(0.35)+np.log(0.4)+np.log(0.03)+np.log(0.7) 

-8.131930790625592

In [10]:
-0.91629+(np.log(0.35) +np.log(0.6) + np.log(0.03) + np.log(0.3))

-7.187468449910586

In [14]:
A_h = np.array([[0.4,0.6],
                [0.7, 0.3]])

B_h = np.array([[0.1, 0.4,0.5],
                [0.4,0.5,0.1]])
pi_h = np.array([0.7, 0.3])
z_h = np.array([1,0,0])
rho_h = np.array([1,1,1])
x_1 = 2

In [16]:
compute_alpha_t(alpha_1_i,A_h,B_h,x_1,z_h,rho_h,n_states=2,n_observations=3)

array([-8.13193079, -7.18746918])

In [17]:
A_h = np.array([[0.4,0.6],
                [0.7, 0.3]])

B_h = np.array([[0.1, 0.4,0.5],
                [0.4,0.5,0.1]])
pi_h = np.array([0.7, 0.3])
z_h = np.array([1,0,0])
rho_h = np.array([1,1,1])
#x_0 = 1
#x_1 = 2
x_vector = [1,2]

In [18]:
T = 20
A_h = np.array([[0.4,0.6],
                [0.7, 0.3]])

B_h = np.array([[0.1, 0.4,0.5],
                [0.4,0.5,0.1]])
pi_h = np.array([0.7, 0.3])
z_h = np.array([1,0,0])
rho_h = np.array([1,1,1])
x_obs = [1,2,0,1,2,1,2,1,0,1,0,1,2,1,0,0,1,0,0,1]
lista_alpha = []
alpha_t_i  = compute_log_alpha_1_i(B_h, x_0, z_h, rho_h, pi_h, n_states = 2, n_observations = 3)
lista_alpha.append(list(alpha_t_i[0]))
for t in range(2,T+1):
    alpha_t_i = compute_alpha_t(alpha_t_i,A_h,B_h,x_obs[1],z_h,rho_h,n_states=2,n_observations=3)
    lista_alpha.append(list(alpha_t_i))

In [20]:
T = 2
A_matrix= np.array([[0.4,0.6],
                    [0.7, 0.3]])
B_matrix = np.array([[0.1, 0.4,0.5],
                     [0.4,0.5,0.1]])
pi_vector = np.array([0.7, 0.3])
z_vector = np.array([[1,0,0],
                    [0,1,0]])
rho_vector = np.array([[1,1,1],
                 [1,1,1]])
x_obs = [1,1]

In [21]:
B_matrix[:,1]

array([0.4, 0.5])

In [22]:
compute_alpha_t_i(A = A_matrix,
                  B =  B_matrix,
                  pi =  pi_vector,
                  z_vector =  z_vector,
                  rho_vector =  rho_vector,
                  x_vector = x_obs,
                  t = 2, 
                  n_states_ = 2, 
                  n_observations_ = 3)

array([[-2.65926004, -2.12026354],
       [-6.96877998, -7.18746918]])

In [23]:
len(x_vector)

2

In [24]:
lista_alpha

[[-2.6592600369327783, -2.120263536200091],
 [-8.355074341939803, -7.410612733098951],
 [-19.341237843845686, -18.396776235004836],
 [-41.31356484765745, -40.36910323881661],
 [-85.258218855281, -84.31375724644015],
 [-173.14752687052805, -172.20306526168724],
 [-348.9261429010222, -347.98168129218135],
 [-700.4833749620105, -699.5389133531696],
 [-1403.597839083987, -1402.6533774751463],
 [-2809.82676732794, -2808.8823057190993],
 [-5622.284623815846, -5621.340162207006],
 [-11247.200336791659, -11246.255875182818],
 [-22497.031762743285, -22496.087301134445],
 [-44996.694614646534, -44995.7501530377],
 [-89996.02031845304, -89995.0758568442],
 [-179994.671726066, -179993.7272644572],
 [-359991.974541292, -359991.0300796832],
 [-719986.580171744, -719985.6357101351],
 [-1439975.791432648, -1439974.846971039],
 [-2879954.2139544557, -2879953.269492847]]